# BAL Pseudobulk analysis, pt. 3

In [1]:
import collections
import math
import os
import sys
import pathlib
import typing

import numpy as np
import pandas as pd
import scanpy as sc
import matplotlib as mpl
import matplotlib.pyplot as plt
import adjustText as adjust_text
import seaborn as sns
import goatools.base
import goatools.obo_parser
import goatools.anno.genetogo_reader
import goatools.goea.go_enrichment_ns
import pybiomart
import statsmodels.stats.multitest

import matplotlib_venn

In [2]:
sys.path.insert(0, "../lib/")

In [3]:
import sc_utils

In [4]:
%config InlineBackend.figure_format = "retina"
pd.options.display.max_columns = 100
plt.rcParams['figure.constrained_layout.use'] = True

In [5]:
DATA = pathlib.Path('../../data/31_bal-object/')

In [6]:
ds_processed = sc.read_h5ad(DATA / '03_bal-object/03_bal-object.h5ad')

In [7]:
DATA_DIR = DATA / 'pseudobulk-gsva'

In [8]:
DEG_DIR = DATA / '11_pseudobulk-v3'

## Apply filtering, GO and plots to all cell types

In [9]:
def plot_ma(filtered_degs, gene_cells, title):
    fig, ax = plt.subplots()
    val_counts = filtered_degs.sign.value_counts(dropna=False).sort_index()
    num_top = min(10, val_counts.get("Up in Control", 0))
    num_bot = min(10, val_counts.get("Up in SSc", 0))
    genes = filtered_degs.loc[
        filtered_degs.padj < 0.05, 
        :
    ].index[list(range(num_top)) + list(range(-num_bot, 0))]

    sign_cells = gene_cells.loc[filtered_degs.index[filtered_degs.sign != ""], :].sum(axis=1)
    min_cells = sign_cells.min()
    max_cells = sign_cells.max()
    colors = {
        "": "gray",
        "Up in Control": lambda x: gene_cells.loc[x.index, :].sum(axis=1),
        "Up in SSc": lambda x: gene_cells.loc[x.index, :].sum(axis=1),
    }
    cmap = {
        "": None,
        "Up in Control": mpl.cm.winter_r,
        "Up in SSc": mpl.cm.autumn_r,
    }
    alphas = {
        "": 0.2,
        "Up in Control": 1,
        "Up in SSc": 1
    }
    processed = []
    for k, v in val_counts.items():
        idx = filtered_degs.sign == k
        kwargs = {}
        label = k
        if label:
            label = f"{label} ({idx.sum()})"
        kwargs["label"] = label
        c = colors[k]
        if callable(c):
            c = c(filtered_degs[idx])
            kwargs["norm"] = mpl.colors.LogNorm(vmin=min_cells, vmax=max_cells)
        kwargs["c"] = c
        kwargs["alpha"] = alphas[k]
        if cmap.get(k):
            kwargs["cmap"] = cmap[k]
        dots = ax.scatter(
            filtered_degs.baseMean[idx], 
            filtered_degs.log2FoldChange[idx],
            s=4,
            **kwargs
        )
        if kwargs.get("norm"):
            fig.colorbar(dots, label="No of cells with gene")
            processed.append(k)
    texts = []
    for r in filtered_degs.loc[genes, :].itertuples(name=None):
        texts.append(ax.text(r[1], r[2], r[0], size=9))
    ax.set_xscale("log")
    ax.figure.set_size_inches(8, 6)
    ax.set_title(title)
    ax.set_ylabel("$log_2$(fold change)", size=16)
    ax.set_xlabel("Mean expression", size=16)
    l = ax.legend(fontsize=12, markerscale=4)
    for lh in l.legendHandles: 
        lh.set_alpha(1)
    fig.axes[0].set_position((0.1, 0.1, 0.72, 0.85))
    for i, d in enumerate(processed):
        if d == "Up in SSc":
            fig.axes[i + 1].set_position((0.84, 0.12, 0.08, 0.375))
        else:
            fig.axes[i + 1].set_position((0.84, 0.52, 0.08, 0.375))
    adjust_text.adjust_text(
        texts, 
        ax=ax, 
        autoalign=False,
        expand_points=(1.5, 1.5),
        force_points=(1, 1),
        arrowprops=dict(arrowstyle='-', color='black', shrinkA=4, shrinkB=4)
    )
    return fig

### Plot heatmap and MA plot for each cell type
Save to output directory

In [10]:
def plot_heatmap(meta, filtered_degs, cell_type, ds_processed):
    groupby = "External Sample ID"
    val_counts = filtered_degs.sign.value_counts(dropna=False)
    num_top = min(20, val_counts.get("Up in Control", 0))
    num_bot = min(20, val_counts.get("Up in SSc", 0))
    genes = filtered_degs.loc[
        filtered_degs.padj < 0.05, 
        :
    ].index[list(range(num_top)) + list(range(-num_bot, 0))]
    
    if num_top + num_bot == 0:
        return None
    
    markers_expr = []
    ds_slice = ds_processed[ds_processed.obs.cell_type == cell_type, :]
    ds_slice = ds_slice[ds_slice.obs[groupby].isin(meta[groupby]), :]
    clusters = sorted(ds_slice.obs[groupby].unique())
    for g in clusters:
        mean_exp = ds_slice.raw.X[
            (ds_slice.obs[groupby] == g).values, 
            :
        ][:, ds_slice.raw.var_names.isin(genes)].mean(axis=0).A.reshape(-1)
        markers_expr.append(mean_exp)

    markers_expr = pd.DataFrame(
        markers_expr, 
        columns=ds_slice.raw.var_names[ds_slice.raw.var_names.isin(genes)], 
        index=clusters
    )
    markers_expr = (markers_expr - markers_expr.min()) / (markers_expr.max() - markers_expr.min())

    axes = sns.clustermap(
        markers_expr.loc[:, genes].T.to_numpy(),
        row_cluster=False,
        cmap="inferno",
        dendrogram_ratio=0.1,
        cbar_pos=(0.05, 0.92, 0.015, 0.05),
        cbar_kws={"ticks": []},
        linecolor="none",
        snap=True,
        figsize=(8, 10),
    )
    ax = axes.ax_heatmap
    ax.set_yticks(pd.Series(range(len(genes))) + 0.5)
    ax.set_yticklabels(
        list(genes), 
        fontstyle="italic",
        fontsize=12,
        rotation=0
    )
    ax.tick_params(
        left=True, labelleft=True, right=False, labelright=False
    )
    ax.set_xticks(pd.Series(range(len(clusters))) + 0.5)
    ax.set_xticklabels(
        [clusters[x] for x in axes.dendrogram_col.reordered_ind], 
        rotation=45, 
        ha="right",
        fontsize=14
    )
    ax.figure.subplots_adjust(0.19, 0.12, 0.76, 0.9)
    axes.cax.set_position((0.8, 0.91, 0.015, 0.05))
    
    trans = mpl.transforms.Affine2D().translate(6, 0)
    for t in ax.get_xticklabels():
        t.set_transform(t.get_transform() + trans)

    ax.figure.text(0.2, 0.98, cell_type, va="top", fontsize=18)
    axes.cax.annotate("row min.", (-2, -0.1), va="top", annotation_clip=False, fontsize=12)
    axes.cax.annotate("row max.", (-2, 1.1), va="bottom", annotation_clip=False, fontsize=12)
    return ax

In [11]:
def plot_cutoffs(gene_cells, n_samples):
    fig, axes = plt.subplots(
        nrows=5, 
        ncols=2, 
        figsize=(10, 8), 
        sharex=True, 
        sharey=True, 
        gridspec_kw={"wspace": 0.03}
    )
    for i in range(10):
        coords = i // 2, i % 2
        ax = axes[coords]
        cnt = (gene_cells > i + 1).sum(axis=1).value_counts().sort_index()
        ax.bar(cnt.index, cnt.values)
        s = "s" if i else ""
        ax.set_title(f"> {i + 1} cell{s} cutoff", pad=-15)
        ax.xaxis.grid(False)
    ax.set_xticks(list(range(n_samples + 1)))
    ax = fig.add_subplot(111, frameon=False)
    ax.tick_params(labelcolor='none', which='both', top=False, bottom=False, left=False, right=False)
    ax.set_title("Distribution of gene expression at different cutoffs", fontsize=18)
    ax.set_xlabel("Number of samples", fontsize=14)
    ax.set_ylabel("Number of genes", labelpad=20, fontsize=14)
    return fig

In [12]:
def _run_go(goeaobj, genes, alpha=0.1, id2names=None):
    TMP_FILE = "__go_temp_file"
    res = goeaobj.run_study(genes, prt=None)
    res_sig = [r for r in res if r.p_fdr_bh < alpha]
    if len(res_sig) == 0:
        return None
    goeaobj.wr_tsv(TMP_FILE, res_sig, itemid2name=id2names)
    res = pd.read_table(TMP_FILE)
    res.drop(["NS", "p_uncorrected", "study_count"], axis=1, inplace=True)
    os.unlink(TMP_FILE)
    return res
    

def run_go(filtered_degs):
    obo_file = goatools.base.download_go_basic_obo(prt=None)
    ass_file = goatools.base.download_ncbi_associations(prt=None)
    obodag = goatools.obo_parser.GODag(obo_file, prt=None)
    objanno = goatools.anno.genetogo_reader.Gene2GoReader(ass_file, prt=None)
    go_ns = {}
    go_ns["BP"] = objanno.get_ns2assc(prt=None)["BP"]
    dataset = pybiomart.Dataset(
        name='hsapiens_gene_ensembl', 
        host='http://oct2014.archive.ensembl.org/'
    ) #found 1878/1898 filtered degs
    genes = dataset.query(attributes=['entrezgene', 'external_gene_name'])
    genes.columns = ["entrez_id", "symbol"]
    genes.set_index("symbol", inplace=True)
    genes = genes.loc[~genes.index.duplicated(), :]
    genes.dropna(inplace=True)
    genes.entrez_id = genes.entrez_id.astype(int)
    id2names = genes.reset_index().set_index("entrez_id").symbol.to_dict()
    found_genes = filtered_degs.index[filtered_degs.index.isin(genes.index)]
    background = genes.loc[found_genes, "entrez_id"]
    goeaobj = goatools.goea.go_enrichment_ns.GOEnrichmentStudyNS(
        background,
        go_ns,
        obodag,
        propagate_counts=False,
        alpha=0.1,
        methods=["fdr_bh"],
        log=None
    )
    upreg_genes = genes.reindex(
        filtered_degs.index[filtered_degs.sign.str.startswith('Up in Control')]
    ).entrez_id
    go_upreg = None
    if upreg_genes.size > 0:
        go_upreg = _run_go(goeaobj, upreg_genes, alpha=0.1, id2names=id2names)
        
    downreg_genes = genes.reindex(
        filtered_degs.index[filtered_degs.sign.str.startswith('Up in SSc')]
    ).entrez_id
    go_downreg = None
    if downreg_genes.size > 0:
        go_downreg = _run_go(goeaobj, downreg_genes, alpha=0.1, id2names=id2names)
    return go_upreg, go_downreg

In [13]:
def get_cell_type_from_slug(slug: str, ds: sc.AnnData) -> str:
    for cell_type in ds.obs.cell_type.unique():
        cell_type_slug = cell_type.replace(" ", "_").replace("/", "_")
        if slug == cell_type_slug:
            return cell_type
    raise ValueError(f"Cannot find cluster for {slug}")

Also, 4 samples sounds like a reasonable cutoff. We can optimize this maybe by requiring at least 2 samples within our comparison groups. Let's try that

## Apply

### 1. Read reference GTF to get gene types

In [20]:
REF_GENOME_GTF = '/projects/b1038/tools/refdata-cellranger-GRCh38-1.2.0/genes/genes.gtf'

In [21]:
ref_genome = open(REF_GENOME_GTF).read().split('\n')

In [28]:
gene_info = []
for line in ref_genome:
    if not line or line[0] == '#':
        continue
    info = line.split('\t')
    line_type = info[2]
    if line_type != 'gene':
        continue
    data = {
        pair.strip().split()[0]: pair.strip().split()[1].strip('"') 
        for pair in info[8].split(';')
        if pair.strip()
    }
    gene_info.append(data)
gene_info = pd.DataFrame(gene_info)

There are few duplicate gene names with conflicting biotypes, but we don't care about this: let's just take all gene names that have `protein_coding` biotype.

In [39]:
protein_coding_genes = gene_info.gene_name[gene_info.gene_biotype.eq('protein_coding')].unique()

### 2. Investigate gene names for filtering

Now let's take all genes from the object, keep only the protein coding ones and see what are common prefixes and suffices?

In [40]:
gene_names = pd.Series(ds_processed.raw.var_names)
gene_names = gene_names[gene_names.isin(protein_coding_genes)]

Let's take genes that have either:
* a sequence of 5 digits in the name
* or a dash in the name

In [41]:
idx = (
    gene_names.str.contains('\d{5,}')
    | gene_names.str.contains('-', regex=False)
)

In [42]:
idx.sum()

429

In [43]:
gene_names_copy = gene_names.str.replace('\d{5,}', '<NUMBERS>')

Top 30 most-common prefixes

In [51]:
gene_names_copy[idx].str[:5].value_counts().head(30)

RP11-    114
AC<NU     62
AL<NU     21
CTD-2     16
HLA-D     13
KRTAP     10
AP<NU     10
MT-ND      7
CTC-4      6
CTD-3      5
LINC<      5
RP5-1      5
CH507      5
XXbac      4
AF<NU      3
CTB-1      3
NKX2-      3
MT-CO      3
RP4-6      3
RP3-4      3
CTB-5      3
CTC-2      2
FO<NU      2
TMEM1      2
NKX6-      2
LA16c      2
FP<NU      2
C1QTN      2
RP5-8      2
RP1-3      2
dtype: int64

Top 20 most-common suffixes

In [45]:
gene_names_copy[idx].str[-4:].value_counts().head(20)

S>.1    54
S>.2    19
S>.3     7
S>.4     6
ERS>     5
S>.6     4
S>.9     3
19.5     3
S>.5     3
19.4     3
12.1     3
-AS1     3
24.4     3
S>.7     3
13.1     3
20.4     2
20.3     2
24.9     2
B8.1     2
21.3     2
dtype: int64

Below is Karolina's code: it processes some prefixes, but not all. Also it is based on newer genome version, and here we have Build 84 aka Cellranger reference 1.2.0

Links:
- https://www.biostars.org/p/9553891/
- https://www.ncbi.nlm.nih.gov/genbank/acc_prefix/
- https://www.biostars.org/p/51456/
- https://www.roswellpark.org/cancertalk/202110/sequencing-human-genome-set-pace-identifying-cancer-it-starts

In [21]:
# Taken from https://github.com/NUPulmonary/Long_COVID/blob/main/21scArches/02scArches-PASC-controls.ipynb
# with modification

# specify the prefix pattern
prefixes_to_remove = ['AC', 'AL', 'AP', 'AF', 'AD', 'BX', 'CR', 'FP', 'KF']

# specify the pattern to match genes to remove
pattern_to_remove = '|'.join(
    [f'{prefix}\d{{6}}\.\d' for prefix in prefixes_to_remove] 
    + ['Z\d{5}\.\d', 'U\d{5}\.\d']
)

Let's finalize our gene exclusions

In [49]:
# Either starts with all this, or ends with -AS1
FUNKY_TRANSCRIPTS = (
    '(^(RP\d{1,2}-|LINC|CT[ABCD]-|AC\d{6}|AL\d{6}'
    '|AP\d{6}|AF\d{6}|XXba|XXya|FO\d{6}|FP\d{6}).+|.+(-AS1)$)'
)

Verify

In [54]:
gene_names[idx & ~gene_names.str.match(FUNKY_TRANSCRIPTS)].str[:4].value_counts().head(20)

HLA-    19
KRTA    10
MT-N     7
CH50     5
TMEM     4
MT-C     4
NKX2     3
NKX3     2
NKX6     2
MT-A     2
ZNF6     2
LA16     2
GS1-     2
CYP3     1
STX1     1
CHKB     1
ZHX1     1
MSAN     1
FAM4     1
CR39     1
dtype: int64

Applying to all cell types and saving additional files

In [57]:
# gene expressed in at least this many cells
THRESHOLD_N_CELLS = 20 

# gene expressed in the above amount of cells in at least this many samples
THRESHOLD_N_SAMPLES_PER_GROUP = 2 

# gene mean expression (counts)
THRESHOLD_MEAN_EXPR = 50

# additionally remove ribosomal genes
REMOVE_GENES_BY_NAME = '^(RPL|RPS).+'
PROTEIN_CODING = protein_coding_genes

MODEL = 'degs-status-sex'

def load_cell_type_info(folder, deg_folder):
    name = folder.name
    degs = pd.read_csv(deg_folder / MODEL / 'degs.csv', index_col=0)
    expr = pd.read_table(folder / 'data' / f'{name}.txt', index_col=0)
    gene_cells = pd.read_table(folder / 'data' / f'{name}-n_cells.txt', index_col=0)
    meta = pd.read_csv(deg_folder / 'meta.csv', index_col=0)

    # ensure we operate on filtered samples
    gene_cells = gene_cells.loc[:, meta["External Sample ID"]]
    gene_cells = gene_cells.loc[gene_cells.sum(axis=1) > 0, :]
    
    return {
        'degs': degs,
        'meta': meta,
        'gene_cells': gene_cells,
        'expr': expr
    }

def filter_genes(info):
    meta = info['meta']
    gene_cells = info['gene_cells']
    expr = info['expr']
    
    genes_to_keep = gene_cells.index
    for group in meta.Status.unique():
        group_samples = meta['External Sample ID'][meta.Status.eq(group)]
        genes_to_keep = np.intersect1d(genes_to_keep, gene_cells.index[
            (
                gene_cells.loc[:, group_samples] >= THRESHOLD_N_CELLS
            ).sum(axis=1) >= THRESHOLD_N_SAMPLES_PER_GROUP
        ])
    genes_to_keep = np.intersect1d(
        genes_to_keep,
        expr.index[expr.mean(axis=1) >= THRESHOLD_MEAN_EXPR]
    )
    return genes_to_keep

def filter_degs(degs, genes_to_keep):
    filtered_degs = degs.loc[degs.index.isin(genes_to_keep), :].copy()
    filtered_degs = filtered_degs.loc[filtered_degs.index.isin(PROTEIN_CODING), :].copy()
    filtered_degs = filtered_degs.loc[~filtered_degs.index.str.match(FUNKY_TRANSCRIPTS), :].copy()
    filtered_degs = filtered_degs.loc[~filtered_degs.index.str.match(REMOVE_GENES_BY_NAME), :].copy()
    # recompute FDR correction on the filtered genes:
    filtered_degs['padj'] = statsmodels.stats.multitest.fdrcorrection(filtered_degs.pvalue)[1]
    # recompute gene status based on new `padj`
    filtered_degs['sign'] = ''
    filtered_degs.loc[
        filtered_degs.padj.lt(0.05) 
        & filtered_degs.log2FoldChange.gt(0), 
        'sign'
    ] = 'Up in Control'
    filtered_degs.loc[
        filtered_degs.padj.lt(0.05) 
        & filtered_degs.log2FoldChange.lt(0), 
        'sign'
    ] = 'Up in SSc'
    return filtered_degs

def save_filtered_degs(folder, filtered_degs):
    filtered_degs.to_csv(folder / MODEL / 'degs-filt.csv')
    filtered_degs.loc[filtered_degs.sign.ne(''), :].to_csv(folder / MODEL / 'degs-sign-filt.csv')
    
    pd.Series(filtered_degs.index[filtered_degs.sign.str.startswith('Up in Control')]).to_csv(
        folder / MODEL / 'degs-sign-control.csv', 
        index=False,
        header=False
    )
    pd.Series(filtered_degs.index[filtered_degs.sign.str.startswith('Up in SSc')]).to_csv(
        folder / MODEL / 'degs-sign-ssc.csv', 
        index=False,
        header=False
    )

def process_cell_type(folder):
    info = {}

    clust = get_cell_type_from_slug(folder.name, ds_processed)
    info.update(load_cell_type_info(DATA_DIR / folder.name, folder))

    degs = info['degs']
    meta = info['meta']
    gene_cells = info['gene_cells']

    genes_to_keep = filter_genes(info)
    filtered_degs = filter_degs(degs, genes_to_keep)
    save_filtered_degs(folder, filtered_degs)
    info["filtered_degs"] = filtered_degs
    
    go_upreg, go_downreg = run_go(filtered_degs)
    if go_upreg is not None:
        go_upreg.to_csv(folder / MODEL / "go-control.csv")
    if go_downreg is not None:
        go_downreg.to_csv(folder / MODEL / "go-ssc.csv")
    info["go_control"] = go_upreg
    info["go_ssc"] = go_downreg

    if filtered_degs.shape[0] > 0:
        fig = plot_ma(filtered_degs, gene_cells, clust)
        fig.savefig(folder / MODEL / "ma-filt.pdf")
        plt.close(fig)
    
    ax = plot_heatmap(meta, filtered_degs, clust, ds_processed)
    if ax:
        ax.figure.savefig(folder / MODEL / "heatmap.pdf")
        plt.close(ax.figure)
    return info

In [58]:
data = {}

for folder in DEG_DIR.iterdir():
    if not folder.is_dir():
        continue
    if not (folder / MODEL).exists():
        continue
    if folder.name == 'global':
        continue
    data[folder.name] = process_cell_type(folder)

**NOTE: DEFAULT TAXID STORED FROM gene2go IS 9606 (human)

HMS:0:01:23.839560 346,070 annotations, 20,758 genes, 18,733 GOs, 1 taxids READ: gene2go 


No handles with labels found to put in legend.



Load BP Gene Ontology Analysis ...
fisher module not installed.  Falling back on scipy.stats.fisher_exact
**NOTE: DEFAULT TAXID STORED FROM gene2go IS 9606 (human)

HMS:0:01:26.911233 346,070 annotations, 20,758 genes, 18,733 GOs, 1 taxids READ: gene2go 

Load BP Gene Ontology Analysis ...
fisher module not installed.  Falling back on scipy.stats.fisher_exact
     15 items WROTE: __go_temp_file


/projects/b1038/Pulmonary/nmarkov/ssc-v2/code/venv/lib/python3.9/site-packages/seaborn/utils.py:95: UserWarning: Calling figure.constrained_layout, but figure not setup to do constrained layout.  You either called GridSpec without the fig keyword, you are using plt.subplot, or you need to call figure or subplots with the constrained_layout=True kwarg.
  fig.canvas.draw()
/projects/b1038/Pulmonary/nmarkov/ssc-v2/code/venv/lib/python3.9/site-packages/seaborn/matrix.py:1203: UserWarning: This figure was using constrained_layout==True, but that is incompatible with subplots_adjust and or tight_layout: setting constrained_layout==False. 
  self._figure.tight_layout(**tight_params)


**NOTE: DEFAULT TAXID STORED FROM gene2go IS 9606 (human)

HMS:0:01:29.347173 346,070 annotations, 20,758 genes, 18,733 GOs, 1 taxids READ: gene2go 

Load BP Gene Ontology Analysis ...
fisher module not installed.  Falling back on scipy.stats.fisher_exact


...

/projects/b1038/Pulmonary/nmarkov/ssc-v2/code/venv/lib/python3.9/site-packages/seaborn/utils.py:95: UserWarning: Calling figure.constrained_layout, but figure not setup to do constrained layout.  You either called GridSpec without the fig keyword, you are using plt.subplot, or you need to call figure or subplots with the constrained_layout=True kwarg.
  fig.canvas.draw()
/projects/b1038/Pulmonary/nmarkov/ssc-v2/code/venv/lib/python3.9/site-packages/seaborn/matrix.py:1203: UserWarning: This figure was using constrained_layout==True, but that is incompatible with subplots_adjust and or tight_layout: setting constrained_layout==False. 
  self._figure.tight_layout(**tight_params)


**NOTE: DEFAULT TAXID STORED FROM gene2go IS 9606 (human)

HMS:0:01:25.597540 346,070 annotations, 20,758 genes, 18,733 GOs, 1 taxids READ: gene2go 

Load BP Gene Ontology Analysis ...
fisher module not installed.  Falling back on scipy.stats.fisher_exact
      2 items WROTE: __go_temp_file


/projects/b1038/Pulmonary/nmarkov/ssc-v2/code/venv/lib/python3.9/site-packages/seaborn/utils.py:95: UserWarning: Calling figure.constrained_layout, but figure not setup to do constrained layout.  You either called GridSpec without the fig keyword, you are using plt.subplot, or you need to call figure or subplots with the constrained_layout=True kwarg.
  fig.canvas.draw()
/projects/b1038/Pulmonary/nmarkov/ssc-v2/code/venv/lib/python3.9/site-packages/seaborn/matrix.py:1203: UserWarning: This figure was using constrained_layout==True, but that is incompatible with subplots_adjust and or tight_layout: setting constrained_layout==False. 
  self._figure.tight_layout(**tight_params)


**NOTE: DEFAULT TAXID STORED FROM gene2go IS 9606 (human)

HMS:0:01:27.617834 346,070 annotations, 20,758 genes, 18,733 GOs, 1 taxids READ: gene2go 

Load BP Gene Ontology Analysis ...
fisher module not installed.  Falling back on scipy.stats.fisher_exact
      4 items WROTE: __go_temp_file


/projects/b1038/Pulmonary/nmarkov/ssc-v2/code/venv/lib/python3.9/site-packages/seaborn/utils.py:95: UserWarning: Calling figure.constrained_layout, but figure not setup to do constrained layout.  You either called GridSpec without the fig keyword, you are using plt.subplot, or you need to call figure or subplots with the constrained_layout=True kwarg.
  fig.canvas.draw()
/projects/b1038/Pulmonary/nmarkov/ssc-v2/code/venv/lib/python3.9/site-packages/seaborn/matrix.py:1203: UserWarning: This figure was using constrained_layout==True, but that is incompatible with subplots_adjust and or tight_layout: setting constrained_layout==False. 
  self._figure.tight_layout(**tight_params)
